<a href="https://colab.research.google.com/github/Shamshad-Gilani/NLP-Projects/blob/main/Word%20Embedding%20using%20Skip%20Gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
import zipfile
import io
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import bigrams
from collections import Counter, defaultdict
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:


# Download and extract the zip file
url = 'https://www.kaggle.com/api/v1/datasets/download/unitednations/un-general-debates'
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall('archive')

# Load the CSV file
csv_file = 'archive/un-general-debates.csv'
data = pd.read_csv(csv_file)

# Preprocess the data
def preprocess(text):
    # Tokenization
    tokens = word_tokenize(text)
    # Lowercasing
    tokens = [token.lower() for token in tokens]
    # Stop Words Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# Combine all text data into a single corpus
corpus = ' '.join(data['text'].tolist())

# Preprocess the corpus
tokens = preprocess(corpus)
print("Preprocessed tokens:", tokens[:50])  # Print the first 50 tokens for reference


Preprocessed tokens: ['\ufeffit', 'inde', 'pleasur', 'member', 'deleg', 'extend', 'ambassador', 'garba', 'sincer', 'congratul', 'elect', 'presid', 'forty-fourth', 'session', 'gener', 'assembl', '.', 'elect', 'high', 'offic', 'well-deserv', 'tribut', 'person', 'qualiti', 'experi', '.', 'fulli', 'confid', 'abl', 'wise', 'leadership', 'assembl', 'consolid', 'gain', 'achiev', 'past', 'year', '.', 'deleg', 'associ', 'previou', 'speaker', 'express', 'appreci', 'dedic', 'effort', 'predecessor', ',', 'excel', 'mr.']


In [19]:
pip install gensim

In [20]:
import gensim
from gensim.models import Word2Vec

# Preprocess the data (Already done)

# Train a Skip Gram model
model = Word2Vec([tokens], vector_size=100, window=5, min_count=1, sg=1)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Function to get the vector of a document
def document_vector(doc):
    doc = preprocess(doc)
    doc_vector = np.mean([model.wv[word] for word in doc if word in model.wv], axis=0)
    return doc_vector

# Vectorize each document in the dataset
data['vector'] = data['text'].apply(document_vector)

# Calculate similarity between documents
similarity_matrix = cosine_similarity(np.stack(data['vector'].values))


In [ ]:
# Find the indices of the top two similar documents
similar_docs = np.unravel_index(np.argsort(similarity_matrix, axis=None)[-3:-1], similarity_matrix.shape)

top_similar_docs = [(data['text'][i], data['text'][j]) for i, j in zip(*similar_docs)]
print("Top two similar documents:", top_similar_docs)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Reduce dimensionality to 2D
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(np.stack(data['vector'].values))

# Plot the embeddings
plt.figure(figsize=(10, 7))
plt.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1])
plt.title('Document Embeddings in 2D Space')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()
